In [1]:
import pandas as pd

In [3]:
from params import engine

dfs = {}


tables = ['user_data', 'post_text_df']

for table in tables:
    dfs[table] = pd.read_sql(f'SELECT * FROM {table}', engine)

In [4]:
category = {}

category['user_data'] = dfs['user_data'].select_dtypes(include='object').columns
category['post_text_df'] = dfs['post_text_df'].select_dtypes(include='object').columns

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

for table in tables:
    for col in category[table]:
        if col == 'text': # обработка колонки "текст"
            continue

        if len(dfs[table][col].unique()) <= 5:
            one_hot = pd.get_dummies(dfs[table][col], prefix=col, drop_first=True).astype(int)
            dfs[table] = pd.concat((dfs[table].drop(col, axis=1), one_hot), axis=1)
        else:
            dfs[table][col] = labelencoder.fit_transform(dfs[table][col])

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# Инициализация TF-IDF векторизатора
tfidf = TfidfVectorizer(max_features=5000)

# Преобразование текста в TF-IDF матрицу (разреженная матрица)
tfidf_matrix = tfidf.fit_transform(dfs['post_text_df']['text'])

# Инициализация PCA
pca = PCA(n_components=15)

# Применение PCA к разреженной матрице TF-IDF
pca_result = pca.fit_transform(tfidf_matrix.toarray())

# Создание DataFrame из результата PCA
pca_df = pd.DataFrame(
    data=pca_result,
    columns=[f'PC{i+1}' for i in range(pca_result.shape[1])],
    index=dfs['post_text_df'].index
)

# Объединение DataFrame с исходными данными
dfs['post_text_df'] = pd.concat([dfs['post_text_df'].drop('text', axis=1), pca_df], axis=1)


In [16]:
for table in tables:
    dfs[table].to_sql(f'david_gusejnov_rbh9686_final_project_{table}', con=engine, if_exists='replace') # записываем таблицу

In [2]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        if "index" in chunk_dataframe.columns:
            chunk_dataframe = chunk_dataframe.drop(columns=["index"])
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)


def load_features() -> pd.DataFrame:
    table_dfs = {}

    tables = ["user_data", "post_text_df"]
    for table in tables:
        query = f"SELECT * FROM david_gusejnov_rbh9686_final_project_{table}"

        table_dfs[table] = batch_load_sql(query)
    return table_dfs

In [3]:
dfs = load_features()

In [4]:
user_info = dfs['user_data'][dfs['user_data']['user_id'] == 554]

In [5]:
user_info

,user_id,gender,age,country,city,exp_group,os_iOS,source_organic
354,554,0,36,7,1032,4,0,0


In [6]:
df = dfs['post_text_df'].merge(user_info, how='cross')
predict_df = df.drop(['user_id', 'post_id'], axis=1)

In [9]:
import os
import pickle


def get_model_path(path: str) -> str:
    if (
        os.environ.get("IS_LMS") == "1"
    ):  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = "/workdir/user_input/model"
    else:
        MODEL_PATH = path
    return MODEL_PATH


def load_models():
    model_path = get_model_path(
        "/home/dayvi/Рабочий стол/final_project/model/catboost_model.pkl"
    )
    model = pickle.load(open(model_path, "rb"))
    return model

model=load_models()

In [10]:
predict_df

,topic,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,...,PC13,PC14,PC15,gender,age,country,city,exp_group,os_iOS,source_organic
0,0,-0.126184,-0.235953,0.036750,-0.088105,-0.137950,0.001769,-0.032578,0.046098,0.021913,...,-0.115062,0.005102,0.136096,0,36,7,1032,4,0,0
1,0,-0.145121,-0.256093,0.032037,0.032826,-0.081603,-0.007729,0.003306,-0.018023,-0.021615,...,0.021649,-0.026606,0.025834,0,36,7,1032,4,0,0
2,0,-0.124699,-0.186698,0.044722,-0.082357,-0.130395,-0.005469,-0.014632,0.022493,0.017248,...,-0.107871,-0.026116,0.069338,0,36,7,1032,4,0,0
3,0,-0.046082,-0.182157,-0.007898,-0.094186,-0.130442,0.001456,0.001112,0.001845,-0.015868,...,-0.066108,0.030243,0.038591,0,36,7,1032,4,0,0
4,0,0.000808,-0.164428,-0.078087,-0.051436,-0.040513,-0.025204,0.014319,-0.002294,-0.036205,...,-0.028075,-0.000490,-0.008332,0,36,7,1032,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,3,-0.096784,0.246096,-0.152090,0.042574,0.022807,0.131586,-0.039134,0.006009,0.044869,...,0.009052,0.045540,-0.025054,0,36,7,1032,4,0,0
7019,3,-0.087600,0.145613,-0.093692,0.024741,-0.101046,0.039890,-0.020475,0.018571,0.011355,...,0.004837,0.063106,-0.026152,0,36,7,1032,4,0,0
7020,3,-0.055415,0.149772,-0.040108,-0.051958,-0.005016,-0.071575,0.039158,-0.013303,-0.076811,...,0.021704,0.112650,-0.028252,0,36,7,1032,4,0,0
7021,3,-0.143932,0.137492,-0.009019,0.011193,0.003255,0.035397,-0.014670,0.008856,-0.018065,...,-0.006104,0.119760,0.064680,0,36,7,1032,4,0,0


In [23]:
df['score'] = model.predict_proba(predict_df)[:, 1]
top_posts = df.sort_values(by='score', ascending=False)

In [22]:
def get_init_dfs():
    df = pd.read_sql(f"SELECT * FROM post_text_df", engine)
    return df

init_post_df = get_init_dfs()

In [25]:
top_posts = init_post_df[init_post_df['post_id'].isin(top_posts['post_id'].head(5))]

In [32]:
for id, text, topic in top_posts.values.tolist():
    print(id, text, topic)

2455 Anyone in the #WestMidlands that has symptoms of #Coronavirus MUST:

• #SelfIsolate immediately
• Book a #COVID19 t… https://t.co/fGk4U5LxR3 covid
3181 The cheapest way to get #Covid19 @AMCTheatres #AMCTheatres https://t.co/NdNm2sDhIy covid
3706 #COVID19: Two new clusters identified in Bukit Tiram and Kuching - Health DG 

https://t.co/mTTuafBaor covid
4135 Anyone in the #EastMidlands that has symptoms of #Coronavirus MUST:

• #SelfIsolate immediately
• Book a #COVID19 t… https://t.co/stkuy6AjLq covid
7098 This is an interesting left turn for Reel 13 Indies. TWO HARBORS is a B&W 75 minute film from Minnesota that features non-actors and is about two people finding a connection through a search for alien life. I applaud the boldness of the Reel 13 programmers of thinking out-of-the-box when selecting this film. I just wish they had picked a stronger film to be bold with. As a matter of fact, I wonder if the choice had more to do with the uniqueness of the film than with the actual 

In [31]:
for i in top_posts.iterrows():
    print(i)

(597, post_id                                                 2455
text       Anyone in the #WestMidlands that has symptoms ...
topic                                                  covid
Name: 597, dtype: object)
(3046, post_id                                                 3181
text       The cheapest way to get #Covid19 @AMCTheatres ...
topic                                                  covid
Name: 3046, dtype: object)
(3550, post_id                                                 3706
text       #COVID19: Two new clusters identified in Bukit...
topic                                                  covid
Name: 3550, dtype: object)
(3957, post_id                                                 4135
text       Anyone in the #EastMidlands that has symptoms ...
topic                                                  covid
Name: 3957, dtype: object)
(6813, post_id                                                 7098
text       This is an interesting left turn for Reel 13 I...
topic